In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.arima.model import ARIMA# ar and ima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller#time series is stationary or not
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [4]:
deliveries=pd.read_csv("/content/drive/MyDrive/Copy of deliveries.csv")

In [5]:
#Handling duplicates in team
team_dup={"Delhi Daredevils":"Delhi Capitals",
"Rising Pune Supergiants":"Rising Pune Supergiant",
"Royal Challengers Bengaluru":"Royal Challengers Bangalore",'Gujarat Lions':'Gujarat Titans'}
deliveries.replace(to_replace=team_dup,inplace=True)

In [6]:
vkohli=deliveries[deliveries.batter=="V Kohli"]
vkohli_score=vkohli.groupby('match_id')['batsman_runs'].sum().reset_index()
vkohli_score_sub=vkohli_score.head(239)
vkohli_score.tail(5)


,match_id,batsman_runs
239,1426290,42
240,1426296,92
241,1426300,27
242,1426306,47
243,1426310,33


In [7]:
S_Dhawan=deliveries[deliveries.batter=="S Dhawan"]
S_Dhawan_score=S_Dhawan.groupby('match_id')['batsman_runs'].sum().reset_index()
S_Dhawan_score_sub=S_Dhawan_score.head(239)
print(S_Dhawan_score)
S_Dhawan_score.tail(5)


     match_id  batsman_runs
0      335984            52
1      335988            25
2      335995             0
3      335998            50
4      336001            19
..        ...           ...
216   1422120            22
217   1422124            45
218   1422129            70
219   1422135             1
220   1426261            14

[221 rows x 2 columns]


,match_id,batsman_runs
216,1422120,22
217,1422124,45
218,1422129,70
219,1422135,1
220,1426261,14


In [16]:
batter=deliveries.groupby(['match_id','batter']).agg({'batsman_runs':'sum'}).reset_index()
batter_scores=batter.groupby('batter')['batsman_runs'].sum().reset_index()
batter_scores=batter_scores.sort_values(by="batsman_runs",ascending=False)
top_batter_scores=batter_scores.head(10)
print(top_batter_scores)
for i in range(len(top_batter_scores)):
  batsman=top_batter_scores.iloc[i]["batter"]
  print(batsman)
  score=batter[batter['batter']==batsman]
  print(score)



             batter  batsman_runs
631         V Kohli          8014
512        S Dhawan          6769
477       RG Sharma          6630
147       DA Warner          6567
546        SK Raina          5536
374        MS Dhoni          5243
30   AB de Villiers          5181
124        CH Gayle          4997
501      RV Uthappa          4954
282      KD Karthik          4843
V Kohli
       match_id   batter  batsman_runs
13       335982  V Kohli             1
56       335985  V Kohli            23
168      335992  V Kohli            13
224      335996  V Kohli            12
252      335998  V Kohli             1
...         ...      ...           ...
16197   1426290  V Kohli            42
16300   1426296  V Kohli            92
16367   1426300  V Kohli            27
16434   1426306  V Kohli            47
16479   1426310  V Kohli            33

[244 rows x 3 columns]
S Dhawan
       match_id    batter  batsman_runs
35       335984  S Dhawan            52
96       335988  S Dhawan            

SARIMA MODEL FOR TOP 10 BATTERS

In [22]:

from sklearn.metrics import mean_absolute_error, mean_squared_error


# Loop through top batters for SARIMA model
for i in range(len(top_batter_scores)):
    batsman = top_batter_scores.iloc[i]["batter"]
    print(f"Batsman: {batsman}")

    # Filter and prepare data
    score = batter[batter['batter'] == batsman]
    score["match_number"] = range(len(score))
    score.set_index("match_number", inplace=True)

   # Splitting into train and test sets
    train_size = int(len(score) * 0.8)  # 80% train, 20% test
    train, test = score["batsman_runs"][:train_size], score["batsman_runs"][train_size:]


    # Fitting the SARIMA model
    model = SARIMAX(
        train,
        order=(1, 1, 1),
        seasonal_order=(1, 1, 1, 12)  # Adjust seasonality if needed
    )
    model_fit = model.fit(disp=False)

     # Forecasting for test set
    test_forecast = model_fit.forecast(steps=len(test))

    # Forecasting beyond the dataset
    future_forecast = model_fit.forecast(steps=5)

    # Evaluation metrics
    mae = mean_absolute_error(test, test_forecast)
    mse = mean_squared_error(test, test_forecast)
    rmse = np.sqrt(mse)

    # Display results
    print("Future Forecasted Values (Next 5 Matches):")
    print(future_forecast)
    print("Evaluation Metrics:")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print("-" * 50)



Batsman: V Kohli


<ipython-input-22-cfc60e6738eb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score["match_number"] = range(len(score))
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
<ipython-input-22-cfc60e6738eb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score["match_number"] = range(len(score))


Future Forecasted Values (Next 5 Matches):
195    20.344845
196    20.112694
197    21.849975
198    21.343810
199    44.497638
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 26.28
Mean Squared Error (MSE): 1087.09
Root Mean Squared Error (RMSE): 32.97
--------------------------------------------------
Batsman: S Dhawan
Future Forecasted Values (Next 5 Matches):
176    23.434634
177    48.532658
178    26.902911
179    34.666220
180    34.341226
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 24.82
Mean Squared Error (MSE): 826.44
Root Mean Squared Error (RMSE): 28.75
--------------------------------------------------
Batsman: RG Sharma


<ipython-input-22-cfc60e6738eb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score["match_number"] = range(len(score))


Future Forecasted Values (Next 5 Matches):
200    28.491598
201    22.300737
202    22.827775
203    39.031439
204    20.767923
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 18.34
Mean Squared Error (MSE): 567.53
Root Mean Squared Error (RMSE): 23.82
--------------------------------------------------
Batsman: DA Warner


...

Future Forecasted Values (Next 5 Matches):
112    33.745147
113    32.158776
114    31.046669
115    40.331099
116    35.301432
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 21.45
Mean Squared Error (MSE): 912.98
Root Mean Squared Error (RMSE): 30.22
--------------------------------------------------
Batsman: RV Uthappa


<ipython-input-22-cfc60e6738eb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score["match_number"] = range(len(score))


Future Forecasted Values (Next 5 Matches):
157    37.162577
158    32.922741
159    31.964321
160    36.268121
161    22.306939
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 20.72
Mean Squared Error (MSE): 597.01
Root Mean Squared Error (RMSE): 24.43
--------------------------------------------------
Batsman: KD Karthik


<ipython-input-22-cfc60e6738eb>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  score["match_number"] = range(len(score))


Future Forecasted Values (Next 5 Matches):
186     8.046327
187    11.438300
188    16.084397
189    12.291907
190    21.426869
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 14.14
Mean Squared Error (MSE): 366.12
Root Mean Squared Error (RMSE): 19.13
--------------------------------------------------


In [23]:
bowler_data = deliveries[['match_id', 'bowler', 'is_wicket', 'total_runs']]


performance = bowler_data.groupby(['match_id', 'bowler']).agg(
    wickets=('is_wicket', 'sum'),
    runs_conceded=('total_runs', 'sum')
).reset_index()

print(performance)

top_bowler = 'YS Chahal'
bowler_timeseries = performance[performance['bowler'] == top_bowler].set_index('match_id')['wickets']
print(bowler_timeseries)

bowler_timeseries = bowler_timeseries.sort_index()

       match_id         bowler  wickets  runs_conceded
0        335982      AA Noffke        1             41
1        335982     AB Agarkar        3             25
2        335982       AB Dinda        2              9
3        335982       CL White        0             24
4        335982       I Sharma        1             13
...         ...            ...      ...            ...
12973   1426312     PJ Cummins        1             18
12974   1426312      SP Narine        1             20
12975   1426312  Shahbaz Ahmed        1             28
12976   1426312    T Natarajan        0             29
12977   1426312       VG Arora        1             26

[12978 rows x 4 columns]
match_id
598029     0
729281     1
729287     2
729299     1
729305     1
          ..
1426294    1
1426299    1
1426303    2
1426310    1
1426311    0
Name: wickets, Length: 159, dtype: int64


In [24]:
bowlers=performance.groupby('bowler')['wickets'].sum().reset_index()
top_bowl=bowlers.sort_values(by="wickets",ascending=False)
top_bowlers=top_bowl.head(10)
top_bowlers

,bowler,wickets
524,YS Chahal,213
119,DJ Bravo,207
348,PP Chawla,201
446,SP Narine,200
355,R Ashwin,198
71,B Kumar,195
438,SL Malinga,188
8,A Mishra,183
193,JJ Bumrah,182
373,RA Jadeja,169


In [25]:
for i in range(len(top_bowlers)):
  bowler=top_bowlers.iloc[i]["bowler"]
  print(bowler)
  bscore=performance[performance['bowler']==bowler]
  print(bscore)

YS Chahal
       match_id     bowler  wickets  runs_conceded
4236     598029  YS Chahal        0             35
4782     729281  YS Chahal        1             18
4816     729287  YS Chahal        2             17
4889     729299  YS Chahal        1             26
4923     729305  YS Chahal        1             17
...         ...        ...      ...            ...
12793   1426294  YS Chahal        1             48
12851   1426299  YS Chahal        1             22
12890   1426303  YS Chahal        2             31
12952   1426310  YS Chahal        1             43
12965   1426311  YS Chahal        0             34

[159 rows x 4 columns]
DJ Bravo
       match_id    bowler  wickets  runs_conceded
84       335989  DJ Bravo        0             37
110      335991  DJ Bravo        0             27
146      335994  DJ Bravo        0             31
182      335997  DJ Bravo        1             25
267      336004  DJ Bravo        1             29
...         ...       ...      ...           

SARIMA MODEL FOR TOP 10 BOWLERS

In [34]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Loop through top bowlers to predict and evaluate
for i in range(len(top_bowlers)):
    bowler = top_bowlers.iloc[i]["bowler"]
    print(f"Bowler: {bowler}")

    # Filter performance data for the bowler
    bscore = performance[performance['bowler'] == bowler]

    # Preparing data
    bscore["match_number"] = range(len(bscore))
    bscore.set_index("match_number", inplace=True)

    # Splitting into train and test sets
    train_size = int(len(bscore) * 0.8)  # 80% train, 20% test
    train, test = bscore["wickets"][:train_size], bscore["wickets"][train_size:]

    # Fitting the SARIMA model
    model = SARIMAX(train, order=(2, 1, 2), seasonal_order=(2, 1, 2, 12))  # Seasonal order parameters (P, D, Q, s)
    model_fit = model.fit()

    # Forecasting for test set
    test_forecast = model_fit.forecast(steps=len(test))

    # Forecasting beyond the dataset
    future_forecast = model_fit.forecast(steps=5)

    # Evaluation metrics
    mae = mean_absolute_error(test, test_forecast)
    mse = mean_squared_error(test, test_forecast)
    rmse = np.sqrt(mse)

    # Results
    print("Future Forecasted Values (Next 5 Matches):")
    print(future_forecast)
    print("Evaluation Metrics:")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
    print("-" * 50)


Bowler: YS Chahal


<ipython-input-34-fee1dd54c0a1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bscore["match_number"] = range(len(bscore))


Future Forecasted Values (Next 5 Matches):
127    1.616692
128    2.548404
129    1.681787
130    2.791349
131    1.648518
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 1.14
Mean Squared Error (MSE): 1.87
Root Mean Squared Error (RMSE): 1.37
--------------------------------------------------
Bowler: DJ Bravo


<ipython-input-34-fee1dd54c0a1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bscore["match_number"] = range(len(bscore))
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


Future Forecasted Values (Next 5 Matches):
126    0.807457
127    0.260087
128    0.632023
129    1.181749
130    1.383175
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 0.93
Mean Squared Error (MSE): 1.34
Root Mean Squared Error (RMSE): 1.16
--------------------------------------------------
Bowler: PP Chawla


...

Future Forecasted Values (Next 5 Matches):
97     2.061281
98     1.214983
99     0.444265
100    1.315695
101    1.417338
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 1.18
Mean Squared Error (MSE): 2.13
Root Mean Squared Error (RMSE): 1.46
--------------------------------------------------
Bowler: A Mishra


<ipython-input-34-fee1dd54c0a1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bscore["match_number"] = range(len(bscore))
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Future Forecasted Values (Next 5 Matches):
129    0.465925
130    0.342694
131    0.412940
132    0.892254
133    0.558841
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 0.92
Mean Squared Error (MSE): 1.32
Root Mean Squared Error (RMSE): 1.15
--------------------------------------------------
Bowler: JJ Bumrah


<ipython-input-34-fee1dd54c0a1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bscore["match_number"] = range(len(bscore))
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
<ipython-input-34-fee1dd54c0a1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

Future Forecasted Values (Next 5 Matches):
106    2.509603
107    2.230513
108    1.187812
109    1.849592
110    3.870949
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 1.77
Mean Squared Error (MSE): 4.02
Root Mean Squared Error (RMSE): 2.00
--------------------------------------------------
Bowler: RA Jadeja
Future Forecasted Values (Next 5 Matches):
168    0.566299
169    0.870959
170    0.351997
171    1.460913
172    0.573179
Name: predicted_mean, dtype: float64
Evaluation Metrics:
Mean Absolute Error (MAE): 0.79
Mean Squared Error (MSE): 0.97
Root Mean Squared Error (RMSE): 0.98
--------------------------------------------------
